# Importing

In [1]:
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
import cv2
from tqdm import tqdm
import datetime
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate 
from tensorflow.keras.layers import Input, Add, Conv2DTranspose
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import clear_output
%matplotlib inline

from IPython.display import HTML
from base64 import b64encode

import os
from PIL import Image

steps:
- read images from paths
- build architecture of unet (https://www.kaggle.com/code/utkarshsaxenadn/breast-cancer-image-segmentation-attention-unet#Encoder)
- compile model, summarize, make sure doesn't exceed 2m params
- loss = meaniou
- predict

# |Read

### Array based

In [7]:
def read_image(folder_path):
   
    def load_image(filename, shape):
        image = tf.keras.preprocessing.image.load_img(filename, target_size=(128,128))
        image = tf.keras.preprocessing.image.img_to_array(image)
        # image = cv2.imread(filename)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image / 255.0  # normalize pixel values to range [0, 1]

    images = np.array([load_image(os.path.join(folder_path, filename), (128,128)) for filename in os.listdir(folder_path)])
            
    return images

In [1]:
test_img_path = '../Fire_Dataset/test/image/'+'fire/'
test_mask_path = '../Fire_Dataset/test/mask/' + 'fire/'
train_img_path = '../Fire_Dataset/train/image/' + 'fire/'
train_mask_path = '../Fire_Dataset/train/mask/'+ 'fire/'
validation_img_path = '../Fire_Dataset/validation/image/'+ 'fire/'
validation_mask_path = '../Fire_Dataset/validation/mask/'+ 'fire/'

In [8]:
# with tf.device('/GPU:0'):
#     train_images = read_image(train_img_path)
#     train_masks = read_image(train_mask_path)
#     validation_images = read_image(validation_img_path)
#     validation_masks = read_image(validation_mask_path)

#     test_images = read_image(test_img_path)
#     test_masks = read_image(test_mask_path)

### Data Generator

In [60]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(img_path, mask_path):
        seed = 2412 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.
        image_datagen = ImageDataGenerator(width_shift_range=0.1,
                                           height_shift_range=0.1,) # custom fuction for each image you can use resnet one too.

        mask_datagen = ImageDataGenerator(width_shift_range=0.1,
                                          height_shift_range=0.1,)  # to make mask as feedable formate (256,256,1)


        train_img =image_datagen.flow_from_directory(img_path,
                                                     target_size=(128,128),
                                                            class_mode=None, seed=seed)

        train_mask = mask_datagen.flow_from_directory(mask_path,
                                                      target_size=(128,128),
                                                      color_mode = 'grayscale',
                                                        class_mode=None, seed=seed)

        train_gen = zip(train_img, train_mask)
        
        for (img, mask) in train_gen:
                img, mask = adjust_data(img, mask)
                yield (img,mask)

        return train_gen
def adjust_data(img,mask):
    img = img / 255
    mask = mask // 255
    
    
    return (img, mask)

In [61]:
test_img_path = '../Fire_Dataset/test/image/'+''
test_mask_path = '../Fire_Dataset/test/mask/' + ''
train_img_path = '../Fire_Dataset/train/image/' + ''
train_mask_path = '../Fire_Dataset/train/mask/'+ ''
validation_img_path = '../Fire_Dataset/validation/image/'+ ''
validation_mask_path = '../Fire_Dataset/validation/mask/'+ ''

In [62]:
train =get_datagen(train_img_path, train_mask_path)
validation =get_datagen(validation_img_path, validation_mask_path)
test =get_datagen(test_img_path, test_mask_path)

# Constructing Unet

In [63]:
# common
import os
import keras
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf

# Data
from tensorflow.keras.utils import to_categorical

# Data Viz
import matplotlib.pyplot as plt

# Model 
from keras.models import Model
from keras.layers import Layer
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import UpSampling2D
from keras.layers import concatenate
from keras.layers import Add
from keras.layers import Multiply
from keras.layers import Input
from keras.layers import MaxPool2D
from keras.layers import BatchNormalization

# Callbacks 
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Metrics
from keras.metrics import MeanIoU

In [84]:
# From: https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py
def dice_coef(y_true, y_pred):
    y_true_f = keras.Flatten(y_true)
    y_pred_f = keras.Flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [85]:
#Build the model
inputs = tf.keras.layers.Input((128, 128, 3))
s = inputs

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef, 'binary_accuracy'])
model.summary()

################################
#Modelcheckpoint
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]



Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_356 (Conv2D)            (None, 128, 128, 16  448         ['input_20[0][0]']               
                                )                                                                 
                                                                                                  
 dropout_151 (Dropout)          (None, 128, 128, 16  0           ['conv2d_356[0][0]']             
                                )                                                          

In [87]:

tf.config.run_functions_eagerly(True)
model_checkpoint = ModelCheckpoint('unet_lung_seg.hdf5', 
                                   monitor='loss', 
                                   verbose=1, 
                                   save_best_only=True)

results = model.fit(train, validation_data = validation, batch_size=16, epochs=25, callbacks=callbacks)


Epoch 1/25


ResourceExhaustedError: Exception encountered when calling layer "conv2d_356" (type Conv2D).

OOM when allocating tensor with shape[32,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

Call arguments received by layer "conv2d_356" (type Conv2D):
  • inputs=tf.Tensor(shape=(32, 128, 128, 3), dtype=float32)

# Training

In [ ]:
next(train)

(array([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          ...,
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          ...,
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.],
          ...,
          [1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],
 
         ...,
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          ...,
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],
 
 
        [